In [1]:
import csv
from itertools import chain
import locale

locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
import re
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_colwidth", None)

In [2]:
PACKAGES_PATH = Path(
    f"/Users/andrewprokhorenkov/CTDS/projects/midrc/indexing-data/packages_acr/packages_acrimage/PETAL"
)
PACKAGES_PATH.mkdir(parents=True, exist_ok=True)

In [3]:
rename_columns = {
    "case_ids": "case_id",
    "series_uid": "series_id",
    "study_uid": "study_id",
    "instance_uid": "instance_id",
}
instances = pd.read_csv("./Open-A1_PETAL_REDCORAL_packaging_manifest.tsv", sep="\t").rename(columns=rename_columns)

In [5]:
list_of_packages = []

for i, row in instances.iterrows():
    case_id = row["case_id"]
    study_id = row["study_id"]
    series_id = row["series_id"]

    series_path = f"./cases/{case_id}/{study_id}/{series_id}.tsv\n"
    if series_path not in list_of_packages:
        list_of_packages.append(series_path)

    # print(f"{case_id} / {study_id}")

    folder = PACKAGES_PATH / "cases" / case_id / study_id

    folder.mkdir(parents=True, exist_ok=True)

    series_file = folder / f"{series_id}.tsv"
    series_file_exist = series_file.exists()

    with open(series_file, mode="a") as f:
        fieldnames = [
            "file_name",
            "file_size",
            "md5sum",
            "case_id",
            "study_id",
            "series_id",
            "instance_id",
            "storage_urls",
        ]
        writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)

        if not series_file_exist:
            writer.writeheader()
        
        to_write = row.to_dict()
        # del to_write["object_id"]
        writer.writerow(to_write)

with open(PACKAGES_PATH / "packages.txt", "w") as f:
    f.writelines(list_of_packages)